In [6]:
from commonfunctions import *
from glob import  glob
import os
from PIL import Image

In [7]:
def inv(img_path, save_path):
    img = io.imread(img_path)
    img = rgb2gray(img)
    bw = (255*(1 - get_thresholded(img, threshold_otsu(img)))).astype(np.uint8)
    io.imsave(save_path, bw)

In [8]:
glob(f'train_data/data/*')

['train_data/data/t44_hw',
 'train_data/data/8b_n',
 'train_data/data/t4',
 'train_data/data/t2',
 'train_data/data/1',
 'train_data/data/chord',
 'train_data/data/#',
 'train_data/data/b_hw',
 'train_data/data/t24_hw',
 'train_data/data/32',
 'train_data/data/dot',
 'train_data/data/8_hw',
 'train_data/data/#_hw',
 'train_data/data/2',
 'train_data/data/8',
 'train_data/data/2_hw',
 'train_data/data/4_hw',
 'train_data/data/b',
 'train_data/data/cross',
 'train_data/data/4',
 'train_data/data/dot_hw',
 'train_data/data/32b_r',
 'train_data/data/t24',
 'train_data/data/16b_n',
 'train_data/data/o',
 'train_data/data/1_hw',
 'train_data/data/16_hw',
 'train_data/data/16',
 'train_data/data/16b_r',
 'train_data/data/t44',
 'train_data/data/clef',
 'train_data/data/8b_r',
 'train_data/data/32b_n',
 'train_data/data/o_hw']

In [9]:
dir_name = 'Eight'
dirs = [path for path in glob(f'train_data/data/*')]
dir_names = [path.split('/')[2] for path in glob(f'train_data/data/*')]
print(dirs)

['train_data/data/t44_hw', 'train_data/data/8b_n', 'train_data/data/t4', 'train_data/data/t2', 'train_data/data/1', 'train_data/data/chord', 'train_data/data/#', 'train_data/data/b_hw', 'train_data/data/t24_hw', 'train_data/data/32', 'train_data/data/dot', 'train_data/data/8_hw', 'train_data/data/#_hw', 'train_data/data/2', 'train_data/data/8', 'train_data/data/2_hw', 'train_data/data/4_hw', 'train_data/data/b', 'train_data/data/cross', 'train_data/data/4', 'train_data/data/dot_hw', 'train_data/data/32b_r', 'train_data/data/t24', 'train_data/data/16b_n', 'train_data/data/o', 'train_data/data/1_hw', 'train_data/data/16_hw', 'train_data/data/16', 'train_data/data/16b_r', 'train_data/data/t44', 'train_data/data/clef', 'train_data/data/8b_r', 'train_data/data/32b_n', 'train_data/data/o_hw']


In [10]:
for i, directory in enumerate(dirs):
    imgs = glob(f'{directory}/*.png')
    for j, img in enumerate(imgs):
        os.rename(img, f'{directory}/{dir_names[i]}_{j}.png')